In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import random
from statistics import mode
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
%config InlineBackend.figure_format = 'retina'
from sklearn.cluster import KMeans
import random
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('sentiment_data/grand_df.csv', index_col=0)
df.head()

,album,anger,anticipation,api_path,disgust,fear,joy,lyrics,negative,polarity,positive,primary_artist,release_date,sadness,song_endpoint,surprise,title,trust,url
0,Boy In Detention (Mixtape),14.0,13.0,/songs/62387,5.0,7.0,12.0,"Beats I be servin', Saks Fifth splurgin' Diam...",11.0,-0.9809,24.0,Chris Brown,2011-08-05,5.0,http://api.genius.com/songs/62387,11.0,100%,17.0,https://genius.com/Chris-brown-100-lyrics
1,Boy In Detention (Mixtape),19.0,1.0,/songs/54007,16.0,15.0,1.0,We in the motherfuckin' buildin' 100 fuckin...,20.0,-0.9967,13.0,Chris Brown,2011-08-05,16.0,http://api.genius.com/songs/54007,1.0,100 Bottles,4.0,https://genius.com/Chris-brown-100-bottles-lyrics
2,X,2.0,4.0,/songs/516349,2.0,1.0,6.0,You got me doing 101 on the 101 Don't go nowh...,6.0,0.9516,11.0,Chris Brown,2014-09-16,2.0,http://api.genius.com/songs/516349,2.0,101 (Interlude),6.0,https://genius.com/Chris-brown-101-interlude-l...
3,None,0.0,0.0,/songs/3103904,0.0,4.0,0.0,Ain't no way we could ignore this fire When I...,3.0,0.8592,4.0,Chris Brown,2017-05-31,2.0,http://api.genius.com/songs/3103904,0.0,10 Feet,2.0,https://genius.com/Chris-brown-10-feet-lyrics
4,None,20.0,17.0,/songs/57217,11.0,12.0,14.0,"Welcome to the Cypher Myself, K-Mac, Ace Hood...",23.0,0.9745,39.0,Chris Brown,2019-04-04,9.0,http://api.genius.com/songs/57217,17.0,2011 BET Cypher Chris Brown and Friends,25.0,https://genius.com/Chris-brown-2011-bet-cypher...


In [3]:
artists = df.groupby('primary_artist')[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'positive', 'negative', 'sadness', 'surprise', 'trust']].mean()

In [4]:
artists = artists.reset_index()
artists

,primary_artist,anger,anticipation,disgust,fear,joy,positive,negative,sadness,surprise,trust
0,2 Chainz,15.120000,11.124000,10.568000,11.580000,10.548000,22.476000,22.200000,10.288000,6.120000,14.492000
1,3OH!3,8.805970,7.029851,6.417910,7.940299,8.343284,16.925373,14.955224,7.298507,5.820896,9.253731
2,5 Seconds of Summer,4.823529,6.852941,4.058824,6.303922,6.696078,13.686275,12.686275,7.656863,3.823529,8.696078
3,50 Cent,19.612069,11.270115,13.408046,15.663793,10.827586,23.442529,29.068966,12.497126,8.862069,15.396552
4,A Great Big World,2.689655,7.827586,2.344828,3.275862,9.862069,16.517241,7.103448,3.793103,4.344828,9.103448
5,A$AP Rocky,15.542857,11.550000,11.871429,13.528571,11.435714,23.150000,25.435714,11.550000,7.742857,15.071429
6,AWOLNATION,5.913043,7.717391,4.565217,7.630435,12.456522,19.108696,11.978261,7.543478,5.717391,11.282609
7,Adam Lambert,5.320755,6.132075,4.811321,8.433962,8.245283,16.226415,12.377358,6.754717,4.622642,9.509434
8,Adele,2.813953,5.813953,2.302326,4.279070,8.302326,15.069767,8.860465,5.023256,2.488372,7.465116
9,Akon,10.884848,9.624242,6.969697,10.236364,11.721212,22.103030,16.975758,9.351515,5.351515,13.066667


In [5]:
def cluster_data(df):
    '''Cluster songs based on their sentiments'''
    pca = PCA(n_components=2)
    pca = pca.fit_transform(df[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'surprise', 'trust', 'positive', 'negative']])
    pca = pd.DataFrame(pca, columns = ['Component1', 'Component2'])
    KMeans_3 = KMeans(n_clusters=3)
    KMeans_3.fit(pca)
    pca['cluster_3'] = KMeans_3.labels_
    KMeans_4 = KMeans(n_clusters=4)
    KMeans_4.fit(pca)
    pca['cluster_4'] = KMeans_4.labels_
    KMeans_5 = KMeans(n_clusters=5)
    KMeans_5.fit(pca)
    pca['cluster_5'] = KMeans_5.labels_
    colors_3 = ["#D1E5f0", "#2166AC", "#F46197"]
    colors_4 = ["#D1E5f0", "#2166AC", "#F46197", "#FE7F2D"]
    colors_5 = ["#D1E5f0", "#2166AC", "#F46197", "#FE7F2D", "#2CA58D"]
    pca['color_3'] = pca['cluster_3'].map(lambda x: colors_3[x])
    pca['color_4'] = pca['cluster_4'].map(lambda x: colors_4[x])
    pca['color_5'] = pca['cluster_5'].map(lambda x: colors_5[x])
#     pca['title'] = df['title']
#     pca['album'] = df['album']
#     pca = pca.replace('None', 'Unknown')
    return pca

In [6]:
clusters = cluster_data(artists)
clusters['artist'] = artists['primary_artist']

In [20]:
clusters['artist'] 

0                 2 Chainz
1                    3OH!3
2      5 Seconds of Summer
3                  50 Cent
4        A Great Big World
5               A$AP Rocky
6               AWOLNATION
7             Adam Lambert
8                    Adele
9                     Akon
10            Alessia Cara
11              Alex Clare
12             Alicia Keys
13              Aloe Blacc
14        American Authors
15           Amy Winehouse
16          Andrea Bocelli
17            Andy Grammer
18           Anna Kendrick
19          Arctic Monkeys
20           Ariana Grande
21           Austin Mahone
22                  Avicii
23           Avril Lavigne
24                  Baauer
25          Bad Meets Evil
26                Bastille
27              Bebe Rexha
28                 Beyoncé
29                Big Sean
              ...         
305           Thomas Rhett
306        Thompson Square
307             Tim McGraw
308              Timbaland
309             Toby Keith
310              Tom Misch
3

In [14]:
# 0 - orange
# 1 - dark blue
# 2 - light blue
# 3 - pink
clusters.cluster_4.value_counts()

2    116
0    113
1     68
3     38
Name: cluster_4, dtype: int64

In [11]:
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import HoverTool, Legend, ColumnDataSource, Panel, Tabs, ColumnDataSource, LabelSet

output_notebook()

TOOLTIPS = """
<div>
    <div width="10px">
        <span style="font-size: 15px;">Name: </span>
        <span style="font-size: 17px; font-weight: bold;">@artist</span>
    </div>
</div>
"""

source = ColumnDataSource(clusters)

p1 = figure(toolbar_location="above", plot_width=900, plot_height=450, tools='pan,box_zoom,reset,save',
            x_axis_label='Principal Component 1', y_axis_label='Principal Component 2')

p1.circle('Component1', 'Component2', size=15, source=source, color='color_4', line_color="black", fill_alpha=0.8, hover_fill_color="black", hover_line_color=None)
p1.background_fill_color = "#f2f2f2"
p1.grid.grid_line_color = "white"
p1.add_tools(HoverTool(tooltips=TOOLTIPS, mode='mouse'))
show(p1)

Loading BokehJS ...